In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [17]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "OTM"

#Product type to hedge: either "CE" or "PE"
# prod_type = "CE"
prod_type = "PE"

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}

#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1


In [18]:
#########################
#User-defined functions #
#########################

def cubic_func(x, a0, a1, a2, a3):
    y = a0 + a1*x + a2*(np.square(x)) + a3 * np.power(x, 3)
    return y

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       batch_size_divisor=10, no_of_epochs = 1000, stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []
    cubic_polyn_volSurf = pd.read_csv(output_path + "A3_" + stock_ident + "_cubicPolyn_vol_surface_params.csv")
    for each_month in mthly_expiries_list:
        strike = dict_mthly_strikes[each_month.strftime("%d-%b-%Y")]
        weekly_pfl_weights = []
        ce_dict = {}
        pe_dict = {}
        cash_dict = {}
        week_list = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")]
        no_of_weeks = len(week_list)
        
        for week in range(0, no_of_weeks):
            s_t = dict_wkly_spots[each_month.strftime("%d-%b-%Y")][week] 
            K = strike          
            r = np.array([dict_wkly_ir[each_month.strftime("%d-%b-%Y")][week]])
            
            if (week == no_of_weeks-1):
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = each_month
            else:
                sim_start_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week]
                sim_end_date = dict_wkly_expiries_each_month[each_month.strftime("%d-%b-%Y")][week + 1]


            df_polyn_date = cubic_polyn_volSurf[cubic_polyn_volSurf['Date'] == sim_start_date.strftime("%d-%b-%Y")]

            # We define Moneyness as (S/K) in vol surface for both calls and puts
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
            
            # EXTRACTING WEEKLY TENOR FOR SIMULATION
            t_list = np.sort(np.unique(np.array(df_polyn_date['T'])))
            tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]
            print(each_month, " Week: ", week)
            print(tenor_list)
            
            if (len(tenor_list) > 2):
                print("There are more than 2 tenors in the vol surface - Please check!")
            
            df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[0]]
            polyn_coeff = df_polyn_coeff[["a0", "a1", "a2", "a3"]].to_numpy().reshape(-1)

            x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)

            if (sim_moneyness <= x_min):
                vol_list = [cubic_func(x_min, *polyn_coeff)]
            elif (sim_moneyness >= x_max):
                vol_list = [cubic_func(x_max, *polyn_coeff)]
            else:
                vol_list = [cubic_func(sim_moneyness, *polyn_coeff)]

            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(dict_ce_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            put_strikes = np.sort(np.array(dict_pe_wkly_strikes[each_month.strftime("%d-%b-%Y")][week])).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
                        
            stock_vec = sim_stock_mat[:, 1]
            mny_list = stock_vec / K
            
            r_f = get_risk_free_rate_from_exact_date(sim_start_date)
            dt = float((each_month - sim_end_date).days) / 365
            
            # Need higher tenor in the first element to avoiding tenor_list[1] as missing when 2 tenors are not available
#             t_list = -np.sort(np.unique(np.array(-df_polyn_date['T'])))
#             tenor_list = [str(int(round(t* 365))) + "D" for t in t_list]            
            
            if (week != no_of_weeks-1):
                df_polyn_coeff = df_polyn_date[df_polyn_date['Tenor'] == tenor_list[1]]
                polyn_coeff = df_polyn_coeff[["a0", "a1", "a2", "a3"]].to_numpy().reshape(-1)
                
                x_min, x_max = df_polyn_coeff[["x_min", "x_max"]].to_numpy().reshape(-1)
            
                vol_arr = cubic_func(mny_list, *polyn_coeff)
                vol_arr[mny_list <= x_min] = cubic_func(x_min, *polyn_coeff)
                vol_arr[mny_list >= x_max] = cubic_func(x_max, *polyn_coeff)
            else:
                vol_arr = np.array([0]) 
                       
            opt_strike = strike

            if (option_type == "CE" and week != no_of_weeks-1):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt)
            elif (option_type == "PE" and week != no_of_weeks-1):
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol_arr, dt) 
            elif (option_type == "CE" and week == no_of_weeks-1):
                option_value = np.maximum(stock_vec - opt_strike, 0) 
            else:
                option_value = np.maximum(opt_strike - stock_vec, 0)
    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 
                     
#             print("SKLearn Linear regression - OLS")
#             lin_model = LinearRegression().fit(x, y)
#             pfl_weights = lin_model.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model.intercept_
#             print("R-Square: ", lin_model.score(x,y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)           
            
#             print("SKLearn Linear regression - Ridge")
#             lin_model_ridge = Ridge(alpha=1.0).fit(x, y)
#             pfl_weights = lin_model_ridge.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model_ridge.intercept_
#             print("R-Square: ", lin_model_ridge.score(x, y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)
            
            
            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)
            
#          
#             #Find Optimal Alpha
#             alphas = np.array([-0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()
            
#             print("Stats Model _ OLS")
#             x = x
#             x = sm.add_constant(x)
#             lin_model2 = OLS(y,x).fit()
#             print(lin_model2.summary())
# #             print(lin_model2.params)
# #           pfl_weights = np.array(lin_model2.params).reshape(-1)
            
            
            ######################################################################################
            ### Portfolio weights of shorter term options from neural network of European options ###
            ######################################################################################
            
            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
            
            
        df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
        df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
        df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

        df_ce_pfl_weights.to_csv(output_path  + "B12A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
        df_pe_pfl_weights.to_csv(output_path + "B12A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
        df_cash_pfl_weights.to_csv(output_path  + "B12A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
    list_of_dates = []
    list_of_prod_type = []
    list_of_strikes = []
    for each_month in dict_mthly_strikes.keys():
        list_of_dates.append(each_month)
        list_of_strikes.append(dict_mthly_strikes[each_month])
        list_of_prod_type.append(prod_type)
    dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
    df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
    df_hedged_prod.to_csv(output_path + "B12A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
    return(df_hedged_prod)


In [19]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

# #Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

# # #Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 

# # # The strike correspnding to the option that is hedged
df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)



2019-08-29  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.9927199997854521
Coefficients:
[-0.00032488 -0.0001896  -0.00279784 -0.         -0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.00399904  0.00126935  0.00138221  0.00049583
  0.00156763  0.00104825  0.          0.00387198]
[1.52250225]
2019-08-29  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9958789419238279
Coefficients:
[-0.00902153 -0.         -0.          0.          0.          0.00225758
  0.00212907  0.          0.          0.          0.          0.
  0.          0.          0.00904094  0.02815355  0.00723393  0.00348202
  0.00373519  0.00335849  0.00110122  0.00356486  0.00687859  0.00219138]
[3.64371054]
2019-08-29  Week:  2
['6D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9958559970497037
Coefficients:
[-0.00365585 -0.00074632 -0.0012

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


R-Square:  1.0
Coefficients:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.]
2019-10-31  Week:  0
['7D', '35D']
SKLearn Linear regression - Lasso
R-Square:  0.9988063658112377
Coefficients:
[-0.0040271  -0.00180473 -0.00167026 -0.         -0.          0.
  0.          0.          0.00255246  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.00434939  0.02972915  0.00689532  0.00537107  0.00433157
  0.0043781   0.00210658  0.00360178  0.00702334  0.00178279  0.00166542
  0.00726364]
[5.74052017]
2019-10-31  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9996792453853418
Coefficients:
[-0.01370046 -0.00173326 -0.         -0.          0.          0.
  0.0040383   0.00153775  0.0011556   0.00193564  0.00131169  0.
  0.          0.          0.          0.          0.0781082   0.0719876
  0.04020643  0.0341501   0.01963263  0.0045

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


SKLearn Linear regression - Lasso
R-Square:  0.9973505015643774
Coefficients:
[-0.00160733 -0.00105301 -0.00233467 -0.         -0.          0.
  0.          0.          0.00068232  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.01417434
  0.00579988  0.00450088  0.00358808  0.00366558  0.00235416  0.00183197
  0.00466021  0.00180189  0.00038242  0.00719389]
[4.20633347]
2019-11-28  Week:  1
['7D', '21D']
SKLearn Linear regression - Lasso
R-Square:  0.9789623570372045
Coefficients:
[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -5.73627442e-04
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  9.82589114e-04
  9.70599385e-0

...

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.574105105954118, tolerance: 7.231716573419726
  positive)


R-Square:  0.994654599062342
Coefficients:
[-3.06546431e-05 -3.28187276e-04 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.30689053e-01  2.02655150e-02  2.40009901e-02  1.35283325e-02
  8.57889172e-03  5.23337336e-03  3.46928674e-03  1.50979068e-03
  5.87386717e-04  8.44699899e-04  7.01730085e-04  3.16857464e-04
  0.00000000e+00  0.00000000e+00  3.41590662e-04  1.52780692e-03]
[0.45131202]
2020-06-25  Week:  3
['7D']
SKLearn Linear regression - Lasso
R-Square:  0.05173750040924319
Coefficients:
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1116.1319235486067, tolerance: 1084.9696185152857
  positive)


R-Square:  0.999961967308083
Coefficients:
[-0.08649526 -0.         -0.          0.0051537   0.00717639  0.00618003
  0.00561605  0.00500153  0.00452929  0.00419573  0.00380898  0.00350257
  0.00300832  0.00271741  0.00413206  0.0039554   0.00737225  0.00074758
  0.          0.          0.          0.          0.          0.
  0.02715138  0.10114022  0.04595417  0.05367804  0.04309678  0.04216176
  0.03144462  0.03364646  0.02579373  0.          0.          0.04126332
  0.07062602  0.02173077  0.00360756]
[69.98388373]
2020-07-30  Week:  1
['7D', '28D']
SKLearn Linear regression - Lasso
R-Square:  0.9996783627205622
Coefficients:
[-0.01865713 -0.00140003 -0.         -0.          0.          0.
  0.00355201  0.00199584  0.00127589  0.00047021  0.00352919  0.00064047
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.00260823
  0.04890515  0.01633158  0.00776854  0.01052916  0.00949288  0.00564427
  0.005534   

/opt/tljh/user/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)
